# __WSI - ćwiczenie 1.__
### __Zagadnienie przeszukiwania i podstawowe podejścia do niego__


1. Narysować funkcje f(x) i g(x).
2. Zaimplementować algorytm najszybszego spadku oraz zastosować go do znalezienia minimum
funkcji f i g.
3. Zbadać wpływ rozmiaru kroku dla różnych (losowych) punktów początkowych.

In [154]:
import numpy as np
from plotly import graph_objs as go
import math
import pandas as pd

RNG = np.random.default_rng()

#### __Definicje funkcji oraz ich gradientów__

In [155]:
def f(vect):
    assert vect.shape == (1,)
    return 10*vect[0]**4 + 3*vect[0]**3 - 30*vect[0]**2 + 10*vect[0]
    
def g(vect):
    assert vect.shape == (2,)
    return 10*vect[1]**4 + 10*vect[0]**4 + 3*vect[0]**3 - 30*vect[0]**2 + 10*vect[0]

def grad_f(vect):
    assert vect.shape == (1,)
    return np.array(40*vect[0]**3 + 9*vect[0]**2 - 60*vect[0] + 10)

def grad_g(vect):
    assert vect.shape == (2,)
    return np.array([40*vect[0]**3 + 9*vect[0]**2 - 60*vect[0] + 10,
                     40*vect[1]**3])

### 

#### __Algorytm najszybszego spadku__


In [156]:
def gradient_descent(start_point, beta, grad, stop_treshlod, num_iters):
    """
    Implementation of gradient descent for minimalisation purposes,
    usable with n-dimensional loss functions, if a suitable gradient 
    function is provided.

    Args:
        start_point: coordinates of the first point
        beta: learning rate of the algorithm = step coefficient
        grad: a function which returns a matrix of partial derivatives of the input function
        stop_treshlod: minimal value of gradient
        num_iters: maximal number of iterations (steps)

    Returns:
        Returns a tuple of an array of points generated by alogrithm
        (dependiong on the gradiant function the point format may vary),
        and a boolean value which depicts whether the function has succesfully
        performed all of its iterations or reached given stop treshold. 

    Raises:
        None
    """
    steps = np.array([start_point])
    point = start_point
    for _ in range(num_iters):
        theta = grad(point)
        # check if algorithm reached a local minimum,
        # i.e. func has flattened out
        # Could be changed for Euclidean distance, 
        # but this should perform faster 
        if abs(theta.sum()) < stop_treshlod:
            break
        # prevent overflow errors
        if abs(theta.sum()) > 100000:
            return steps, False
        # perform algorithm step
        point = point - beta * theta
        # add point to the output array
        steps = np.append(steps, [point], 0)
    return steps, True


#### __Generowanie wykresów__


Wykres dla funkcji jednej zmiennej f(x):

In [157]:
max_r = 3
X = np.linspace(-max_r, max_r, 100)
Y = np.array([f(np.array([x])) for x in X])

pt = RNG.uniform(-max_r, max_r, 1)
dsc = gradient_descent(pt, 0.005, grad_f, 0.01, 100)
if not dsc[1]:
    print("Algorithm hasn't found the optimum, steps are out of bounds")
else:
    steps = np.array([x for x in dsc[0]
                    if abs(x[0]) < max_r])
    XS = steps[:, 0]
    YS = np.array([f(x) for x in steps])

    layout = go.Layout(width=700, height=500,
                       title='Gradient Descent of single variable function',
                       xaxis_title='x',
                       yaxis_title='f(x)',
                       plot_bgcolor='DarkSeaGreen')
    fig = go.Figure(data=[go.Scatter(x=X, y=Y, line=dict(color='DarkSlateGrey', width=3))], 
                    layout=layout)
    fig.add_trace(go.Scatter(x=XS, y=YS, mode='lines+markers', 
                            marker=dict(size=6, color=YS,
                            colorscale='Agsunset'),
                            line=dict(color='DarkSlateGrey', width=1)))
    fig.show()

Wykres dla funkcji dwóch zmiennych g(x, y):

In [158]:
max_r = 3
l = np.linspace(-max_r, max_r, 100)
X, Y = l, l
Z = np.array([[g(np.array([x, y])) for x in X] for y in Y])

pt = RNG.uniform(-max_r, max_r, 2)
dsc = gradient_descent(pt, 0.005, grad_g, 0.1, 100)
if not dsc[1]:
    print("Algorithm hasn't found the optimum, steps are out of bounds")
else:
    steps = np.array([pt for pt in dsc[0] if abs(pt[0]) < max_r and abs(pt[1]) < max_r])
    XS, YS = steps[:,0], steps[:,1]
    ZS = np.array([g(np.array([x, y])) for x, y in zip(XS, YS)])

    layout = go.Layout(width = 700, height =700,
                       title_text='Gradient Descent minimalisation of double variable function',
                       xaxis_title='x',
                       yaxis_title='y')                       
    fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z, colorscale='Emrld',
                                    opacity=0.5)], layout=layout)
    fig.update_traces(contours_z=dict(show=True, usecolormap=True))
    fig.add_scatter3d(x=XS, y=YS, z=ZS, mode='lines+markers', 
                      marker=dict(size=4, color=ZS,               
                                  colorscale='Agsunset'),
                      line=dict(color='DarkSlateGrey', width=0.7))
    fig.update_layout(scene=dict(zaxis_title="g(x, y)"))
    fig.show()

#### __Analiza wydajności funkcji najszybszego spadku w zależności od wartości współczynnika kroku__

Z przeprowadzonych obserwacji wywnioskowałem, że potrzebuję co najmniej kilkuset iteracji (uruchomień dla punktów początkowych) by zbadać generalne zachowanie algorytmu. Badania przeprowadzałem głównie na obszarze od -3 do 3, na którym 100 kroków algorytmu było zdecydowanie wystarczające żeby funkcja zatrzymała się po osiągnięciu dolnego limitu wartości gradientu, oznaczonego współczynnikiem odcięcia.


In [159]:
vals = [g(p) for p in dsc[0]]
layout = go.Layout(width=700, height=500,
                title_text='g(x,y) function value for GDA steps',
                plot_bgcolor='DarkSeaGreen',
                xaxis_title='Traversed point number',
                yaxis_title='g(x, y)')
fig = go.Figure(layout=layout)
fig.add_trace(go.Scatter(mode='lines+markers', 
                         marker=dict(size=6,
                                     color='DarkSlateGrey'),
                         line=dict(color='DarkSlateGrey', width=1), 
                         x=list(range(len(vals))), y=vals))
fig.show()

Współczynnik odcięcia __stop_treshold__ ustawiłem na 0.01, co wydało mi się wystarczające żeby stwierdzić że funkcja wystarczajaco zbliżyła się do minimum lokalnego i się stamtąd nie ruszy.

Funkcja testująca współczynnik kroku __beta__ uruchamia algorytm gradientowy __num_iters__ razy dla losowych punktów w zakresie badanego obszzaru i zwraca listę punktów końcowych wywołań algorytmu.

In [160]:
def test_beta(func, func_parameter_num, grad, beta: int, max_r, num_iters: int):
    end_points = []
    for _ in range(num_iters):
        p = RNG.uniform(max_r[0], max_r[1], func_parameter_num)
        dsc = gradient_descent(p, beta, grad, 0.01, 100)
        if not dsc[1]:
            end_points.append(np.nan)
        else:
            end_points.append(func(dsc[0][-1]))
    return np.array(end_points)

Metodą inżynierską zbadałem, że optymalna wartość zmienia się w zależnosci od rozimaru powierzchni dopuszczalnej. Dlatego, oraz dlatego, że w przypadkach innych funkcji których wykresów nie znamy, korzystne będzie przebadać większy zakres współczynników __beta__. W tym celu tworzę _dataframe_ który posłuży mi do dalszej analizy. Każda kolumna owej tabeli odpowiada badanym współczynnikom __beta__, a w wierszach znajdują się informacje o wartościach zwracanych przez _GDA_ dla losowych  punktów startowych.

In [161]:
b_arr = np.arange(0.0001, 0.02, 0.0001)
num_iters = 100
performance = {np.round(b, 4):(test_beta(g, 2, grad_g, b, [-10, 10], num_iters)) for b in b_arr}

df = pd.DataFrame(columns=[b for b in performance.keys()])
for col in df.columns:
    df[col] = performance[col]

In [162]:
df

,0.0001,0.0002,0.0003,0.0004,0.0005,0.0006,0.0007,0.0008,0.0009,0.0010,...,0.0190,0.0191,0.0192,0.0193,0.0194,0.0195,0.0196,0.0197,0.0198,0.0199
0,-36.708349,-33.732825,-41.638588,-6.134383,-6.529886,NaN,-41.480204,NaN,-6.832500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.006029,NaN,NaN
1,-39.226819,-3.091017,-5.458436,-6.163532,-42.062219,-6.652042,-42.379146,-6.815073,-42.627515,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-28.706674,-38.115128,-41.103049,-41.760063,-6.418967,-42.346450,NaN,NaN,NaN,-6.862228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-41.814506,NaN,NaN
3,6.214393,-3.131213,-5.469306,-41.743356,-6.439053,-6.615236,NaN,-42.412011,-6.884330,-42.495872,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.487848,-3.157018,-5.418767,-6.271249,-6.725661,NaN,-6.717443,-41.844393,NaN,-42.489244,...,NaN,NaN,-32.07052,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,11.541579,-39.873528,-41.156154,-6.235562,-6.473332,-42.238249,NaN,-42.433701,-42.458691,-42.488726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,12.307580,-3.793820,-5.396995,-2.231075,-42.476814,-42.424481,-6.200015,NaN,-42.453214,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,11.821083,-3.048135,-41.083792,-6.997448,-42.096331,-6.645878,-6.728978,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,11.885478,-39.078643,-41.192129,-6.132926,-42.084780,-4.782315,-42.339183,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
df.describe()

,0.0001,0.0002,0.0003,0.0004,0.0005,0.0006,0.0007,0.0008,0.0009,0.0010,...,0.0190,0.0191,0.0192,0.0193,0.0194,0.0195,0.0196,0.0197,0.0198,0.0199
count,100.000000,100.000000,100.000000,100.000000,100.000000,85.000000,73.000000,62.000000,58.000000,44.000000,...,7.000000,2.000000,5.000000,4.000000,4.000000,3.000000,0.0,8.000000,6.000000,5.000000
mean,-8.545962,-17.300200,-22.357735,-23.935908,-22.139935,-27.866694,-26.267812,-24.036662,-23.412567,-21.456072,...,-7.546953,-32.174334,-17.118828,-11.864726,-21.182683,-17.481908,NaN,-23.508503,-11.238222,-19.920163
std,19.891911,19.304854,18.035933,17.956282,17.857798,17.708421,17.848843,17.946708,17.883443,17.715959,...,0.923822,0.664723,13.223975,8.849511,17.301483,19.385683,NaN,17.527808,6.787352,17.833946
min,-41.919921,-42.240141,-42.404667,-42.627610,-42.602350,-42.595288,-42.620084,-42.627320,-42.627678,-42.610273,...,-8.899295,-32.644364,-32.070520,-25.105361,-42.218924,-39.851595,NaN,-41.824811,-22.477791,-42.555901
25%,-27.106596,-39.098191,-41.148057,-41.774599,-42.076952,-42.256840,-42.350317,-42.408724,-42.458536,-42.489687,...,-7.952798,-32.409349,-31.102497,-12.532451,-31.929648,-23.428775,NaN,-41.046299,-14.447101,-36.028079
50%,-0.817263,-4.020237,-6.875256,-21.539688,-6.788863,-42.234832,-42.334446,-7.004365,-6.978730,-6.918071,...,-7.005974,-32.174334,-8.409267,-7.673797,-17.752931,-7.005954,NaN,-21.310780,-7.006096,-7.006302
75%,11.588504,-3.115272,-5.450057,-6.159619,-6.472425,-6.640277,-6.733727,-6.788969,-6.835566,-6.867281,...,-7.005940,-31.939319,-7.005938,-7.006072,-7.005966,-6.297065,NaN,-7.006011,-7.005988,-7.006029
max,12.341622,60.952830,1.145669,-0.428986,1.392587,-2.879903,-6.200015,-5.955302,-6.828149,-6.794109,...,-7.005924,-31.704304,-7.005918,-7.005948,-7.005948,-5.588175,NaN,-7.004928,-7.005961,-7.004502


Następnie rysuję wykres wartości funkcji dla punktów zwracanych przez _GDA_, dla odpowiednich wartosci współczynników __beta__. 

In [164]:
layout = go.Layout(width=700, height=500,
                   title_text='Algorithm outputs in function of beta value',
                   plot_bgcolor='DarkSeaGreen',
                   xaxis_title='beta',
                   yaxis_title='min(g(x, y)) from GDA')
fig = go.Figure(layout=layout)
for _, row in df.iterrows():
    fig.add_trace(go.Scatter(mode='markers', marker_color='DarkSlateGrey', 
                             opacity=0.5, x=df.columns, y=row, showlegend = False))
fig.show()

Na tym wykresie nie widać jednak ile iteracji dla każdej wartości __beta__ nie osiągneło minimum. Wobec tego następny wykres przedstawia stosunek ilości nieudanych iteracji do wykonanych.

In [165]:
fail_rate = df.isna().sum()/num_iters

layout = go.Layout(width=700, height=500,
                title_text='Beta fail (out of bounds) rate',
                plot_bgcolor='DarkSeaGreen',
                xaxis_title='beta',
                yaxis_title='failed/total iterations')
fig = go.Figure(data=[go.Bar(x=df.columns, y=fail_rate, marker_color='DarkSlateGrey')], layout=layout)
fig.show()

Następnie po ustaleniu maksymalnego błędu wyznaczam najlepszy parametr __beta__, taki, który daje możliwie najniższe wyniki _GDA_.

In [166]:
max_error = 0.1
tmp_df = df[df.columns[df.isna().sum()/num_iters < max_error]]
best_b = tmp_df.columns[tmp_df.sum().argmin()]
print(best_b)

0.0004


Na koniec rysuję wykres wartości osiaganych przez _GDA_ dla znalezionego parametru __beta__. 

In [167]:
layout = go.Layout(width=700, height=500,
                   title_text='Best beta outputs',
                   plot_bgcolor='DarkSeaGreen',
                   xaxis_title='Iteration number',
                   yaxis_title='min(g(x, y)) from GDA')
fig = go.Figure(data=[go.Bar(y=df[best_b], marker_color='DarkSlateGrey')], layout=layout)
fig.update_yaxes(autorange="reversed")
fig.show()

Z powyższego wykresu można ostatecznie wnioskować, że funkcja ta ma dwa minima lokalne. Dla znalezionego parametru beta powinniśmy wobec tego otrzymać wartość minimalną w okolicy _-42_ bądź _-7_.

In [168]:
p = RNG.uniform(-10, 10, 2)
g_min = gradient_descent(p, best_b, grad_g, 0.0001, 1000)[0][-1]
print(g_min)
print(g(g_min))

[1.01255865 0.17618216]
-6.996687249061111


Analogiczny proces przeprowadziłem dla funkcji jednej zmiennej f(x):

In [169]:
b_arr = np.arange(0.0001, 0.02, 0.0001)
num_iters = 100
performance = {np.round(b, 4):(test_beta(f, 1, grad_f, b, [-10, 10], num_iters)) for b in b_arr}

df = pd.DataFrame(columns=[b for b in performance.keys()])
for col in df.columns:
    df[col] = performance[col]
max_error = 0.1
tmp_df = df[df.columns[df.isna().sum()/num_iters < max_error]]
best_b = tmp_df.columns[tmp_df.sum().argmin()]
print(best_b)

0.0004


In [171]:
p = RNG.uniform(-10, 10, 1)
f_min = gradient_descent(p, best_b, grad_f, 0.0001, 1000)[0][-1]
print(f_min)
print(f(f_min))

[1.01255746]
-7.006322161189814


Ostatecznie, przy _10000_ iteracji dla funkcji _g(x, y)_ algorytm znalajduje minima: _g([-1.41, 0.05]) = -42.63_ oraz _g([1.01 0.05]) = -7.01_, a także dla funkcji f(x): _f([-1.41]) = -42.63_ i 
 _f([1.01]) = -7.01_, gdzie to pierwsze to minima globalne.

#### Wnioski


Ze względu na pewną ułomność tego algorytmu, oraz naturę badanych funkcji, algorytm ma tendencję wpadać w minima lokalne, a także wystrzeliwywać wyniki w kosmos kiedy pochodne badanych funkcji są duże oraz gwałtownie maleją w okolicy minimum. Najbezpieczniej wobec tego byłoby wybrać możliwie najmniejszy parametr __beta__ i możliwie najwięcej iteracji, co oczywiście nie jest możliwe jeśli zależny nam na czasie. Problem dotyczący wpadania w minima lokalne można próbować rozwiązać stosując stochastyczny algorytm spadku gradientu, który zamiast używać dokładnie obliczonego gradientu, korzysta z aproksymowanego gradientu który posiada przez to losowe zakłócenia. Ze względu na to że algorytm nie porusza się wtedy dokładnie w kierunku spadku wartości funkcji optymalizowanej, istnieje szansa, że lokalne minima zostaną ominięte. 